In [1]:
import duckdb
from dotenv import load_dotenv


load_dotenv()

conn = duckdb.connect()


In [2]:
!du -sh mirrulations/bulk/raw-data/


2.5G	mirrulations/bulk/raw-data/


In [3]:
!du -sh parquet/


424M	parquet/


### Dockets

In [4]:
query = """\
SELECT *
FROM read_parquet(
  'parquet/dockets/agency_code=*/year=*/*.parquet',
  hive_partitioning = 1
)
LIMIT 10;
;
"""

conn.query(query)


┌───────────────┬───────────────┬─────────────────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

### Documents

In [5]:
query = """\
SELECT *
FROM read_parquet(
  'parquet/documents/agency_code=*/year=*/*.parquet',
  hive_partitioning = 1
)
LIMIT 10;
;
"""

conn.query(query)


┌───────────────┬────────────────────┬──────────┬───────────────┬─────────────────────┬─────────────────────┬─────────────────────┬─────────────────────┬─────────────────────┬────────────┬───────────┬────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

### Comments

In [6]:
query = """\
SELECT *
FROM read_parquet(
  'parquet/comments/agency_code=*/year=*/docket_id=*/*.parquet',
  hive_partitioning = 1
)
LIMIT 10;
;
"""

conn.query(query)


┌────────────────────┬──────────────────────────────┬────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬───────────────────┬─────────────────────┬─────────────────────┬─────────────────────┬────────────────┬────────────────────────────────┬───────────┬──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

## Querying from S3

In [7]:
query = """\
CREATE OR REPLACE SECRET secret (
    TYPE s3,
    PROVIDER credential_chain,
    PROFILE 'civictechdc'
);
"""
conn.query(query)


┌─────────┐
│ Success │
│ boolean │
├─────────┤
│ true    │
└─────────┘

In [8]:
query = """\
SELECT COUNT(*)
FROM read_parquet(
        's3://jay-mirrulations-demo/mirrulations_parquet/comments/agency_code=*/year=*/docket_id=*/*.parquet',
        hive_partitioning = 1
    )
;
"""

conn.query(query)


┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│       479462 │
└──────────────┘

In [9]:
# Example power of SQL

# Query comments for records where the `comment` field contains the word "supports"

query = """\
SELECT docket_id, comment_id, *
FROM read_parquet(
        's3://jay-mirrulations-demo/mirrulations_parquet/comments/agency_code=*/year=*/docket_id=*/*.parquet',
        hive_partitioning = 1
    )
WHERE LOWER(comment) LIKE '%supports%'
LIMIT 10;
"""

conn.query(query)


┌───────────────┬────────────────────┬────────────────────┬──────────────────────┬──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [10]:
# Count whether `comment` field contains the word "supports"

query = """\
WITH comments AS (
    SELECT
        *,
        LOWER(comment) LIKE '%supports%' AS contains_supports
    FROM read_parquet(
        's3://jay-mirrulations-demo/mirrulations_parquet/comments/agency_code=*/year=*/docket_id=*/*.parquet',
        hive_partitioning = 1
    )
)
SELECT comments.contains_supports, COUNT(*)
FROM comments
GROUP BY comments.contains_supports
"""

conn.query(query)


┌───────────────────┬──────────────┐
│ contains_supports │ count_star() │
│      boolean      │    int64     │
├───────────────────┼──────────────┤
│ true              │         5101 │
│ NULL              │         2404 │
│ false             │       471957 │
└───────────────────┴──────────────┘

In [11]:
# Example complex query with joins

# Count comments by Docket Type and whether the comment text contains the word "supports"

query = """\
WITH dockets AS (
    SELECT *
    FROM read_parquet(
        's3://jay-mirrulations-demo/mirrulations_parquet/dockets/agency_code=*/year=*/*.parquet',
        hive_partitioning = 1
    )
),
comments AS (
    SELECT
        *,
        LOWER(comment) LIKE '%supports%' AS contains_supports
    FROM read_parquet(
        's3://jay-mirrulations-demo/mirrulations_parquet/comments/agency_code=*/year=*/docket_id=*/*.parquet',
        hive_partitioning = 1
    )
)
SELECT dockets.agency_code, dockets.docket_type, comments.contains_supports, COUNT(*)
FROM comments
JOIN dockets ON comments.docket_id = dockets.docket_id
GROUP BY dockets.agency_code, docket_type, comments.contains_supports
ORDER BY dockets.agency_code, docket_type, comments.contains_supports
"""

conn.query(query)


┌─────────────┬───────────────┬───────────────────┬──────────────┐
│ agency_code │  docket_type  │ contains_supports │ count_star() │
│   varchar   │    varchar    │      boolean      │    int64     │
├─────────────┼───────────────┼───────────────────┼──────────────┤
│ CMS         │ Nonrulemaking │ false             │       130021 │
│ CMS         │ Nonrulemaking │ true              │         4562 │
│ CMS         │ Nonrulemaking │ NULL              │           16 │
│ CMS         │ Rulemaking    │ false             │      5225340 │
│ CMS         │ Rulemaking    │ true              │        41243 │
│ CMS         │ Rulemaking    │ NULL              │          141 │
│ DEA         │ Nonrulemaking │ false             │         1770 │
│ DEA         │ Nonrulemaking │ true              │            1 │
│ DEA         │ Rulemaking    │ false             │      3281337 │
│ DEA         │ Rulemaking    │ true              │        86601 │
│ DEA         │ Rulemaking    │ NULL              │          3

In [12]:
# Get the mean difference between comments.posted_date and comments.receive_date
# But only where the docket type is 'Rulemaking' and the comment contains the word "supports"

query = """\
WITH comments AS (
    SELECT
        *,
        LOWER(comment) LIKE '%supports%' AS contains_supports
    FROM read_parquet(
        's3://jay-mirrulations-demo/mirrulations_parquet/comments/agency_code=*/year=*/docket_id=*/*.parquet',
        hive_partitioning = 1
    )
),
dockets AS (
    SELECT *
    FROM read_parquet(
        's3://jay-mirrulations-demo/mirrulations_parquet/dockets/agency_code=*/year=*/*.parquet',
        hive_partitioning = 1
    )
)
SELECT mean(comments.posted_date - comments.receive_date)
FROM comments
JOIN dockets ON comments.docket_id = dockets.docket_id
WHERE dockets.agency_code = 'CMS'
    AND dockets.docket_type = 'Rulemaking'
    AND comments.contains_supports = TRUE
GROUP BY comments.contains_supports
"""

conn.query(query)


┌──────────────────────────────────────────────────────────┐
│ mean(("comments".posted_date - "comments".receive_date)) │
│                         interval                         │
├──────────────────────────────────────────────────────────┤
│ 4 days 21:56:38.373862                                   │
└──────────────────────────────────────────────────────────┘

In [13]:
query = """\
EXPLAIN (ANALYZE, FORMAT TEXT)

WITH comments AS (
    SELECT
        *,
        LOWER(comment) LIKE '%supports%' AS contains_supports
    FROM read_parquet(
        's3://jay-mirrulations-demo/mirrulations_parquet/comments/agency_code=*/year=*/docket_id=*/*.parquet',
        hive_partitioning = 1
    )
),
dockets AS (
    SELECT *
    FROM read_parquet(
        's3://jay-mirrulations-demo/mirrulations_parquet/dockets/agency_code=*/year=*/*.parquet',
        hive_partitioning = 1
    )
)
SELECT mean(comments.posted_date - comments.receive_date)
FROM comments
JOIN dockets ON comments.docket_id = dockets.docket_id
WHERE dockets.agency_code = 'CMS'
    AND dockets.docket_type = 'Rulemaking'
    AND comments.contains_supports = TRUE
GROUP BY comments.contains_supports
"""

print(conn.execute(query).fetchone()[1])


┌─────────────────────────────────────┐
│┌───────────────────────────────────┐│
││    Query Profiling Information    ││
│└───────────────────────────────────┘│
└─────────────────────────────────────┘
EXPLAIN (ANALYZE, FORMAT TEXT)  WITH comments AS (     SELECT         *,         LOWER(comment) LIKE '%supports%' AS contains_supports     FROM read_parquet(         's3://jay-mirrulations-demo/mirrulations_parquet/comments/agency_code=*/year=*/docket_id=*/*.parquet',         hive_partitioning = 1     ) ), dockets AS (     SELECT *     FROM read_parquet(         's3://jay-mirrulations-demo/mirrulations_parquet/dockets/agency_code=*/year=*/*.parquet',         hive_partitioning = 1     ) ) SELECT mean(comments.posted_date - comments.receive_date) FROM comments JOIN dockets ON comments.docket_id = dockets.docket_id WHERE dockets.agency_code = 'CMS'     AND dockets.docket_type = 'Rulemaking'     AND comments.contains_supports = TRUE GROUP BY comments.contains_supports 
┌───────────────────────

In [14]:
!find "parquet/comments" -type f -name '*.parquet' | wc -l


680
